In [8]:
import keras
from keras import layers

target_shape = (200, 200)

In [12]:
fst_sent_in = keras.Input(shape=(None,))
scd_sent_in = keras.Input(shape=(None,))

embd_func_1 = layers.Embedding(len(relation2id), 300)

fst_sent_embd = embd_func_1(fst_sent_in)
scd_sent_embd = embd_func_1(scd_sent_in)

lstm_func_1 = layers.Bidirectional(layers.LSTM(150, return_sequences=True))
lstm_func_2 = layers.Bidirectional(layers.LSTM(150, return_sequences=False))

fst_lstm_mid = lstm_func_1(fst_sent_embd)
scd_lstm_mid = lstm_func_1(scd_sent_embd)

fst_lstm_out = lstm_func_2(fst_lstm_mid)
scd_lstm_out = lstm_func_2(scd_lstm_mid)

#fst_redu_mean = tf.reduce_mean(fst_lstm_out, axis=1)
#scd_redu_mean = tf.reduce_mean(scd_lstm_out, axis=1)



In [ ]:
# Input layer, using integer to represent each relation type
#note that inputs_path is the path inputs, while inputs_out_re is the output relation inputs
inputs_path = keras.Input(shape=(None,), dtype="int32")
inputs_out_re = keras.Input(shape=(None,), dtype="int32")

# Embed each integer in a 300-dimensional vector as input
in_embd = layers.Embedding(len(relation2id), 300)(inputs_path)

# Embed each integer in a 300-dimensional vector as output
ot_embd = layers.Embedding(len(relation2id), 300)(inputs_out_re)

#add 2 layer bi-directional LSTM
mid_lstm = layers.Bidirectional(layers.LSTM(150, return_sequences=True))(in_embd)
out_lstm = layers.Bidirectional(layers.LSTM(150, return_sequences=True))(mid_lstm)

#sum into an embedding
sum_lstm = tf.reduce_sum(out_lstm, axis=1)

#remove the time dimension from the output embd since there is only one step
sum_ot_embd = tf.reduce_sum(ot_embd, axis=1)

#concatenate the lstm output and output embd
concat = layers.concatenate([sum_lstm, sum_ot_embd], axis=1)

#add the dense layer
dense_1 = layers.Dense(32, activation='relu')(concat)
batch_norm = layers.BatchNormalization()(dense_1)
dropout = layers.Dropout(0.5)(batch_norm)

#final layer
final_out = layers.Dense(1, activation='sigmoid')(dropout)

#put together the model
model = keras.Model([inputs_path, inputs_out_re], final_out)

#compile the model
model.compile(
    loss='binary_crossentropy',
    optimizer="Adam",
    metrics=["accuracy"],
)

In [ ]:
#function for filling big-batch lists
def fill_in_big_batch_lists(existing_ids, x_p_lists, x_r_lists, y_lists, len_record,
                            half_len, path_set, lower_bd, upper_bd, Class_2, one_hop, s_t_r, 
                            relation2id, entity2id, id2relation, id2entity, epoch):

    carry_on = True
    total_count = 0
    slot_checkpoint = -1
    
    while carry_on:

        source_id = random.choice(existing_ids)
        
        result, length_dict = Class_2.obtain_paths('direct_neighbour', source_id, 
                                                   'not_specified', lower_bd, upper_bd, one_hop)

        for target_id in result:

            if not carry_on:
                break

            #we want to make sure that the same position in different path-len list stores
            #path between the same node pair. Hence, in each result, we want to obtain the
            #minimum path amount across all different lengths, so that this amount of paths
            #will be appended to lists across different lengths.
            minimun_amount = 0

            for length in length_dict:

            #for positive mode, (source_id, target_id) guaranteed to be in s_t_r
            if (source_id, target_id) in s_t_r:
                dir_r = list(s_t_r[(source_id, target_id)])

            for path in result[target_id]:

                if not carry_on:
                    break

                #update the connection dictionary all the time ^_^
#                if mode == 'positive':
#                    connect_dict[path].update(s_t_r[(source_id, target_id)])

                path_len = len(path)

                if (path_len >= lower_bd) and (len_record[path_len] < half_len) and (
                    path not in path_set[path_len]):

                    slot = deepcopy(len_record[path_len])
                    
                    if mode == 'positive':
                        
                        x_p_lists[path_len].append(path)
                        
                        #under this case, dir must exist
                        relation_id = random.choice(dir_r)
                        x_r_lists[path_len].append(relation_id)
                        
                        y_lists[path_len].append(1.)
                        
                        len_record[path_len] += 1
                        path_set[path_len].add(path)
                    
                    else:
                        
                        selectable = {i for i in id2relation}
                        
#                        if path in connect_dict:
#                            selectable = selectable.difference(connect_dict[path])
                            
                        if (source_id, target_id) in s_t_r:
                            selectable = selectable.difference(s_t_r[(source_id, target_id)])
                        
                        if len(selectable) > 0:
                            x_p_lists[path_len].append(path)
                            
                            relation_id = random.choice(list(selectable))
                            x_r_lists[path_len].append(relation_id)
                            
                            y_lists[path_len].append(0.)
                            
                            len_record[path_len] += 1
                            path_set[path_len].add(path)
                
                #here we use the slot_checkpoint to avoid repatitive printing counting result
                if len_record[length] % 500 == 0 and len_record[length] != slot_checkpoint:

                    print('generating the', mode, 'half', length, len_record[length], half_len, 'for epoch', epoch)

                    slot_checkpoint = len_record[length]

                if len_record[length] >= half_len:

                    carry_on = False

        total_count += 1

        if total_count >= 3*len(entity2id):

            carry_on = False

In [ ]:
#function to build all the big batches
def build_big_batches(holder_len, connect_dict, lower_bd, upper_bd, Class_2, one_hop, s_t_r, 
                      relation2id, entity2id, id2relation, id2entity, epoch):
    
    if holder_len % 10 != 0:
        raise ValueError('We would like to take 10X as a big-batch size')
    
    #we first append the positive half, then negative half. After that we shuffle the list
    half_len = int(holder_len/2)
    
    #90% to be train, 10% to be validation
    train_len = 9*int(holder_len/10)
    valid_len = int(holder_len/10)
    
    #first create the train holder arrays
    x_p_train = {length: np.zeros((train_len, length), dtype=int) for length in range(lower_bd, upper_bd+1)} 
    x_r_train = {length: np.zeros((train_len, 1), dtype=int) for length in range(lower_bd, upper_bd+1)}
    y_train = {length: np.zeros((train_len,)) for length in range(lower_bd, upper_bd+1)}

    #first create the validation holder arrays
    x_p_valid = {length: np.zeros((valid_len, length), dtype=int) for length in range(lower_bd, upper_bd+1)} 
    x_r_valid = {length: np.zeros((valid_len, 1), dtype=int) for length in range(lower_bd, upper_bd+1)}
    y_valid = {length: np.zeros((valid_len,)) for length in range(lower_bd, upper_bd+1)}
    
    #when filling in big-batch for the first time, we use list instead of the arrays directly
    x_p_lists = {length: [] for length in range(lower_bd, upper_bd+1)} 
    x_r_lists = {length: [] for length in range(lower_bd, upper_bd+1)}
    y_lists = {length: [] for length in range(lower_bd, upper_bd+1)}
    
    #create a record list recording the current slot for each half batch (pos/neg)
    pos_record = {length: 0 for length in range(lower_bd, upper_bd+1)}
    neg_record = {length: 0 for length in range(lower_bd, upper_bd+1)}
    
    #for validation case: not all entities in entity2id are in one_hop, 
    #so we need to find out who are in
    existing_ids = set()
    
    for s_1 in one_hop:
        existing_ids.add(s_1)
        
    existing_ids = list(existing_ids)

    #start filling by reading the ObtainPathsByDynamicProgramming
    #here, we start from the longest length and drop the length iteratively
    #in this way, lower length will be pre-filled in in previous iterations for higher lengths
    #so the previous running will not be wasted and the entire filling speed is much faster
    for length in range(upper_bd, lower_bd-1, -1):
        
        #we want to remove duplicated paths in the big-batch, 
        #so use path_set to keep unique ones
        path_set = defaultdict(set)
        
        fill_in_big_batch_lists('positive', existing_ids, x_p_lists, x_r_lists, y_lists, pos_record,
                                half_len, connect_dict, path_set, lower_bd, length, Class_2, one_hop, s_t_r, 
                                relation2id, entity2id, id2relation, id2entity, epoch)
        
        path_set = defaultdict(set)
        
        fill_in_big_batch_lists('negative', existing_ids, x_p_lists, x_r_lists, y_lists, neg_record,
                                half_len, connect_dict, path_set, lower_bd, length, Class_2, one_hop, s_t_r, 
                                relation2id, entity2id, id2relation, id2entity, epoch)

    #shuffle the list and fill-in the arrays
    for length in range(upper_bd, lower_bd-1, -1):
        
        if (len(x_p_lists[length]) != len(x_r_lists[length])) or (
            len(x_r_lists[length]) != len(y_lists[length])) or (
            len(x_p_lists[length]) != pos_record[length] + neg_record[length]):
            
            raise ValueError('list of the same path-length does not in the same size')
        
        id_list = [i for i in range(holder_len)]
        
        random.shuffle(id_list)
                
        for i in range(train_len):
            
            ID = id_list[i]
            
            path = deepcopy(x_p_lists[length][ID])
            relation_id = deepcopy(x_r_lists[length][ID])
            y_label = deepcopy(y_lists[length][ID])
            
            x_p_train[length][i] = np.asarray(path)
            x_r_train[length][i] = relation_id
            y_train[length][i] = y_label
            
        for i in range(valid_len):
            
            ID = id_list[train_len + i]
            
            path = deepcopy(x_p_lists[length][ID])
            relation_id = deepcopy(x_r_lists[length][ID])
            y_label = deepcopy(y_lists[length][ID])
            
            x_p_valid[length][i] = np.asarray(path)
            x_r_valid[length][i] = relation_id
            y_valid[length][i] = y_label
                
    return(x_p_train, x_r_train, y_train, x_p_valid, x_r_valid, y_valid, pos_record, neg_record)

In [21]:
import numpy as np

test = [[1, 2],[3, 4],[5, 6]]
test_ar = np.asarray(test[1:], dtype='int')

test_ar

array([[3, 4],
       [5, 6]])

In [8]:
test = {i for i in {1:2, 2:3, 3:4}}
test

{1, 2, 3}

In [ ]:
    #######################################
    ###do the training#####################
    for path_len in pos_record:
        
        if (pos_record[path_len] == int(holder_len/2)) and (
            neg_record[path_len] == int(holder_len/2)):
            
            print('training on length', path_len, upper_bd, 'for epoch', entire_epoch)
        
            model.fit([x_p_train[path_len], x_r_train[path_len]], y_train[path_len], 
                      validation_data=([x_p_valid[path_len], x_r_valid[path_len]], y_valid[path_len]),
                      batch_size=8, epochs=3)

In [15]:
Dict = {1: []}
Dict[1].append([1,2,3] + [4]*10)
Dict

{1: [[1, 2, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]]}

In [ ]:
for key in x_p_lists:
    print(key, len(x_p_lists[key]['s']), len(x_p_lists[key]['l']), 
          len(y_list[key]),  len(x_r_list[key]))

In [ ]:
x_p_lists[(2,4)]

In [ ]:
#load the test dataset
one_hop_test = dict() 
data_test = set()
s_t_r_test = defaultdict(set)

len_0 = len(relation2id)
size_0 = len(entity2id)

#fill in the sets and dicts
Class_1.load_train_data('../data/fb237_v4_ind/train.txt', 
                        one_hop_test, data_test, s_t_r_test,
                        entity2id, id2entity, relation2id, id2relation)

Class_1.load_train_data('../data/fb237_v4_ind/valid.txt', 
                        one_hop_test, data_test, s_t_r_test,
                        entity2id, id2entity, relation2id, id2relation)

Class_1.load_train_data('../data/fb237_v4_ind/test.txt', 
                        one_hop_test, data_test, s_t_r_test,
                        entity2id, id2entity, relation2id, id2relation)

len_1 = len(relation2id)
size_1 = len(entity2id)

if len_0 != len_1:
    raise ValueError('unseen relation!')

In [ ]:
def relation_ranking(s, t, lower_bd, upper_bd, one_hop, id2relation, model):
    
    path_holder = defaultdict(set)
    
    for iteration in range(15):
    
        result, length_dict = Class_2.obtain_paths('target_specified', 
                                                   s, t, lower_bd, upper_bd, one_hop)
        if t in result:
            
            for path in result[t]:
                
                path_holder[len(path)].add(path)
                
    score_dict = defaultdict(float)
    
    for length in path_holder:
        
        for Tuple in path_holder[length]:
            
            list_a = list()
            list_b = list()
            
            for i in range(len(id2relation)):

                list_a.append(list(Tuple))
                list_b.append([i])
                
            input_a = np.array(list_a)
            input_b = np.array(list_b)
                
            pred = model.predict([input_a, input_b], verbose = 0)
            
            for i in range(pred.shape[0]):

                score_dict[i] += pred[i]

    return(score_dict)

In [ ]:
########################################################
#obtain the precision-recall area under curve (AUC-PR)##

#randomly select 10% of the triples
selected = random.sample(list(data_test), int(len(data_test)/10))

###Hit at 1#############################
#generate the negative samples by randomly replace relation with all the other relaiton
Hits_at_1 = 0

for i in range(len(selected)):
    
    s_true, r_true, t_true = selected[i][0], selected[i][1], selected[i][2]
    
    score_dict = relation_ranking(s_true, t_true, 5, 12, one_hop_test, id2relation, model)
    
    #[... [score, r], ...]
    temp_list = list()
    
    for r in id2relation:
        
        if r in score_dict:
            
            temp_list.append([score_dict[r], r])
        
    temp_list.sort(key = lambda x: x[0], reverse=True)
    
    p = 0
    inverse_r = 0
    exist_tri = 0
    
    while p < len(temp_list) and temp_list[p][1] != r_true:
        
        #we want to see how many inverse relaiton are ranked above true relation
        #then, we remove them from ranking, otherwise it is not fair for us to compare our
        #result with other model who does not consider inverse relations
        if temp_list[p][1] % 2 != 0:
            
            inverse_r += 1
        
        #moreover, we want to remove existing triples
        elif ((s_true, temp_list[p][1], t_true) in data_test) or (
              (s_true, temp_list[p][1], t_true) in data):
            
            exist_tri += 1
            
        p += 1
        
    print('Calculating', p - inverse_r - exist_tri, i, len(selected))
    
#    if r_true == temp_list[0][1]:
        
#        Hits_at_1 += 1
        
#    print('Calculating hit at 1', Hits_at_1/(i+1), i, len(selected))

**Model before Jan 2, 2023**

In [ ]:
# Input layer, using integer to represent each relation type
#note that inputs_path is the path inputs, while inputs_out_re is the output relation inputs
fst_path = keras.Input(shape=(None,), dtype="int32")
scd_path = keras.Input(shape=(None,), dtype="int32")

#the relation input layer (for output embedding)
id_rela = keras.Input(shape=(None,), dtype="int32")

# Embed each integer in a 300-dimensional vector as input,
# note that we add another "space holder" embedding, 
# which hold the spaces if the initial length of two paths are not the same
in_embd_var = layers.Embedding(len(relation2id)+1, 300)

# Obtain the embedding
fst_p_embd = in_embd_var(fst_path)
scd_p_embd = in_embd_var(scd_path)

# Embed each integer in a 300-dimensional vector as output
rela_embd = layers.Embedding(len(relation2id)+1, 300)(id_rela)

#add 2 layer bi-directional LSTM
lstm_layer_1 = layers.Bidirectional(layers.LSTM(150, return_sequences=True))
lstm_layer_2 = layers.Bidirectional(layers.LSTM(150, return_sequences=False))

#first LSTM layer
fst_lstm_mid = lstm_layer_1(fst_p_embd)
scd_lstm_mid = lstm_layer_1(scd_p_embd)

#second LSTM layer
fst_lstm_out = lstm_layer_2(fst_lstm_mid)
scd_lstm_out = lstm_layer_2(scd_lstm_mid)

#concatenate the output vector from both siamese tunnel
path_concat = layers.concatenate([fst_lstm_out, scd_lstm_out], axis=-1)

#remove the time dimension from the output embd since there is only one step
sum_r_embd = tf.reduce_sum(rela_embd, axis=1)

#concatenate the lstm output and output embd
concat = layers.concatenate([path_concat, sum_r_embd], axis=-1)

#add the dense layer
dense_1 = layers.Dense(32, activation='relu')(concat)
batch_norm = layers.BatchNormalization()(dense_1)
dropout = layers.Dropout(0.25)(batch_norm)

#final layer
final_out = layers.Dense(2, activation='softmax')(dropout)

#put together the model
model = keras.Model([fst_path, scd_path, id_rela], final_out)

**Jan 4, 2023**

In [ ]:
holder_len = 20000
lower_bd = 2
upper_bd = 10
diff = 2
entire_epochs = 20
each_epoch = 2

#90% to be train, 10% to be validation
train_len = 9*int(holder_len/10)

current = 0

for entire_epoch in range(entire_epochs):
    
    ######################################
    ###pre-define the lists###############
    
    #define the lists
    x_p_list, x_p_dict, x_r_dict, y_dict, path_comb = list(), dict(), dict(), dict(), dict()
    filled, total_num_need_to_fill = set(), 0

    #build the lists first
    for i in range(lower_bd, upper_bd+1):

        Max = min(upper_bd, i+diff)

        for j in range(i, Max+1):
            
            x_p_list.append((i,j))
            x_p_dict[(i,j)] = {'s': [], 'l': []}
            x_r_dict[(i,j)] = list()
            y_dict[(i,j)] = list()
            path_comb[(i,j)] = set()
            total_num_need_to_fill += 1
    
    #######################################
    ###build the big-batches###############
    
    #here is a tricky thing: if the distance between upper_bd and lower_bd is big,
    #then when run path_finding_dynamic_programming, 
    #the chance to obtain paths with length close to lower bd is low.
    #For instance, if lower_bd = 2, upper_bd = 10, then most found paths will have length
    #from 6 to 10. Paths with length 2 or 3 is relatively rare.
    #Hence, we need to first build big-batches for combination between shorter lengths.
    #otherwise the while loop may last for really really long time,
    #due to difficulty to find shorter paths.
    build_big_batches(diff, holder_len, lower_bd, lower_bd, Class_2, one_hop, s_t_r,
                          x_p_dict, x_r_dict, y_dict, path_comb, filled, total_num_need_to_fill,
                          relation2id, entity2id, id2relation, id2entity, entire_epoch)    
    
    if upper_bd - lower_bd > 3:
        
        build_big_batches(diff, holder_len, lower_bd, lower_bd + 3, Class_2, one_hop, s_t_r,
                          x_p_dict, x_r_dict, y_dict, path_comb, filled, total_num_need_to_fill,
                          relation2id, entity2id, id2relation, id2entity, entire_epoch)        
    
    #fill in the training array list
    build_big_batches(diff, holder_len, lower_bd, upper_bd, Class_2, one_hop, s_t_r,
                          x_p_dict, x_r_dict, y_dict, path_comb, filled, total_num_need_to_fill,
                          relation2id, entity2id, id2relation, id2entity, entire_epoch)
    
    #######################################
    ###do the training#####################
    for key in x_p_list:
        
        #generate the input arrays
        x_train_s = np.asarray(x_p_dict[key]['s'][:train_len], dtype='int')
        x_train_l = np.asarray(x_p_dict[key]['l'][:train_len], dtype='int')
        x_train_r = np.asarray(x_r_dict[key][:train_len], dtype='int')
        y_train = np.asarray(y_dict[key][:train_len], dtype='int')
        
        x_valid_s = np.asarray(x_p_dict[key]['s'][train_len:], dtype='int')
        x_valid_l = np.asarray(x_p_dict[key]['l'][train_len:], dtype='int')
        x_valid_r = np.asarray(x_r_dict[key][train_len:], dtype='int')
        y_valid = np.asarray(y_dict[key][train_len:], dtype='int')
        
        print('training on length', key, 'for epoch', entire_epoch)
        
        #we use a bit dummy method: in order to make sure the model is saved,
        #we save it at each length pair in each epoch!!!
        #so unless it is the first pair first epoch, we will always read from previous checkpoint
        if entire_epoch != 0 or key != x_p_list[0]:
            
            print('load model')
            
            model = keras.models.load_model('../weight_bin/' + model_name + '.h5')
            
        else:
            
            #compile the model
            model.compile(
                loss='categorical_crossentropy',
                optimizer=opt,
                metrics=["categorical_accuracy"],)

        model.fit([x_train_s, x_train_l, x_train_r], y_train, 
                  validation_data=([x_valid_s, x_valid_l, x_valid_r], y_valid),
                  batch_size=4, epochs=current+each_epoch, initial_epoch=current)
        
        current += each_epoch
        
        # Save model and weights
        add_h5 = model_name + '.h5'
        save_dir = os.path.join(os.getcwd(), '../weight_bin')

        if not os.path.isdir(save_dir):
            os.makedirs(save_dir)
        model_path = os.path.join(save_dir, add_h5)
        model.save(model_path)
        print('Save model')
        del(model)
        
        del(x_train_s, x_train_l, x_train_r, y_train)
        del(x_valid_s, x_valid_l, x_valid_r, y_valid)
        
    del(x_p_list, x_p_dict, x_r_dict, y_dict, path_comb)
    del(filled, total_num_need_to_fill)

**Jan 4, 2023**

In [ ]:
#function to build all the big batches
def build_big_batches(diff, holder_len, lower_bd, upper_bd, Class_2, one_hop, s_t_r,
                      x_p_dict, x_r_dict, y_dict, path_comb, filled, total_num_need_to_fill,
                      relation2id, entity2id, id2relation, id2entity, epoch):
    
    if holder_len % 10 != 0:
        raise ValueError('We would like to take 10X as a big-batch size')
    
    #the set of all relation IDs
    relation_id_set = set()
    for i in range(len(id2relation)):
        
        if i not in id2relation:
            raise ValueError('error when generaing id2relation')
        
        relation_id_set.add(i)
    
    num_r = len(id2relation)
    
    #count how many appending has performed
    count = 0
    
    #we count how many combinations need to fill
    need_to_fill = set()
    
    for i in range(lower_bd, upper_bd+1):

        Max = min(upper_bd, i+diff)

        for j in range(i, Max+1):
            
            need_to_fill.add((i,j))

    #in case not all entities in entity2id are in one_hop, 
    #so we need to find out who are indeed in
    existing_ids = set()
    
    for s_1 in one_hop:
        existing_ids.add(s_1)
        
    existing_ids = list(existing_ids)
    
    carry_on = True
    
    while carry_on:

        #obtain paths by dynamic programming
        source_id = random.choice(existing_ids)

        result, length_dict = Class_2.obtain_paths('direct_neighbour', source_id, 
                                                   'not_specified', lower_bd, upper_bd, one_hop)
        for target_id in result:

            if not carry_on:
                break
            
            #we want to make sure s, t are indeed directly connected, 
            #otherwise there is no relation for positive sample
            #also, we want to make sure s and t and not connected by all relations, 
            #although this situation is rare. 
            #But in that case, there is no relation for negative samples 
            if ((source_id, target_id) in s_t_r) and (
                len(s_t_r[(source_id, target_id)]) < len(id2relation)):
                
                dir_r = list(s_t_r[(source_id, target_id)])
                
                non_dir_r = list(relation_id_set.difference(dir_r))
                
                if len(dir_r) <= 0:
                    
                    raise ValueError('errors when creating s_t_r !!')
                
                #iterate over path_1
                for path_1 in result[target_id]:

                    if not carry_on:
                        break

                    #iterate over path_2
                    for path_2 in result[target_id]:

                        if not carry_on:
                            break

                        #decide which path is shorter and which is longer
                        if len(path_1) <= len(path_2):

                            path_s, path_l = path_1, path_2

                        else:

                            path_s, path_l = path_2, path_1                            

                        #whether lengths of the two paths satisfies the requirments
                        if (len(path_s) >= lower_bd) and (len(path_l) <= upper_bd) and (
                            abs(len(path_s)-len(path_l)) <= diff):

                            #further consider: whether the corresponding length comb is not full,
                            #and whether this path pair is new, and whether the two paths are different
                            #But it is optional to require the path to be new. 
                            #We may remove this requirment, especially for short paths
                            if (len(y_dict[(len(path_s), len(path_l))]) < holder_len) and (
                                (path_s, path_l) not in path_comb[(len(path_s), len(path_l))]) and (
                                path_s != path_l):
                                
                                #we always add one positive and one negative situation together,
                                #hence, the length of list should always be even.
                                #also we want to make sure the length of lists coincide
                                if (len(x_p_dict[(len(path_s), len(path_l))]['s']) != len(
                                    y_dict[(len(path_s), len(path_l))])) or (
                                    len(x_p_dict[(len(path_s), len(path_l))]['s']) != len(
                                        x_p_dict[(len(path_s), len(path_l))]['l'])) or (
                                    len(y_dict[(len(path_s), len(path_l))]) != len(
                                        x_r_dict[(len(path_s), len(path_l))])) or (
                                    len(y_dict[(len(path_s), len(path_l))]) % 2 != 0):
                                    
                                    raise ValueError('error when building big batches: length error')
                                
                                #####positive#####################
                                relation_id = random.choice(dir_r)
                                
                                #append the paths: note that we add the space holder id at the end
                                #of the shorter path
                                x_p_dict[(len(path_s), len(path_l))]['s'].append(
                                          list(path_s) + [num_r]*abs(len(path_s)-len(path_l)))
                                x_p_dict[(len(path_s), len(path_l))]['l'].append(list(path_l))

                                #append relation
                                x_r_dict[(len(path_s), len(path_l))].append([relation_id])
                                y_dict[(len(path_s), len(path_l))].append([1., 0.])
                                
                                #####negative#####################
                                relation_id = random.choice(non_dir_r)
                                
                                #append the paths: note that we add the space holder id at the end
                                #of the shorter path
                                x_p_dict[(len(path_s), len(path_l))]['s'].append(
                                          list(path_s) + [num_r]*abs(len(path_s)-len(path_l)))
                                x_p_dict[(len(path_s), len(path_l))]['l'].append(list(path_l))

                                #append relation
                                x_r_dict[(len(path_s), len(path_l))].append([relation_id])
                                y_dict[(len(path_s), len(path_l))].append([0., 1.])
                                
                                ######add to path combinations#####
                                #here is the tricky part: we have to add both (path_s, path_l)
                                #and (path_l, path_s). This is because when the length are the same
                                #adding only one situation won't guarantee that 
                                #the same path with different order is also considered.
                                #in other words: path combination don't have order, but our dict does.
                                #so we have to add both situations.
                                path_comb[(len(path_s), len(path_l))].add((path_s, path_l))
                                path_comb[(len(path_s), len(path_l))].add((path_l, path_s))
                                
                                count += 1
                                
                                if count % 10000 == 0:
                                    print('generating big-batches', count, 
                                          int(holder_len*0.5*len(need_to_fill)), 'in epoch', epoch)
                                
                            if len(y_dict[(len(path_s), len(path_l))]) >= holder_len:
                                
                                prev_size = len(filled)

                                filled.add((len(path_s), len(path_l)))
                                
                                post_size = len(filled)
                                
                                #when we indeed find a new filled combo, we will print, which looks like:
                                #big-batches 1 ( 2 2 ) in N completed in epoch k
                                #big-batches 2 ( 3 5 ) in N completed in epoch k
                                #big-batches 3 ( 3 4 ) in N completed in epoch k
                                if post_size > prev_size:

                                    print('big-batches', len(filled), 
                                          '(', len(path_s), len(path_l), ')',
                                          'in', total_num_need_to_fill, 
                                          'completed in epoch', epoch)
                        
                        #check whether to finish
                        if len(need_to_fill.difference(filled)) == 0:
                            
                            carry_on = False